In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openai
import pickle
import logging
import traceback
from time import sleep, time
from datetime import datetime
from openai import OpenAIError
from data.serialize import SerializerSettings
from models.utils import grid_iter
from models.promptcast import get_promptcast_predictions_data
from models.llmtime import get_llmtime_predictions_data
from models.validation_likelihood_tuning import get_autotuned_predictions_data

# Set up API key and environment
openai.api_key = os.getenv('OPENAI_API_KEY', "")

def create_logger(filename: str, name: str = None):
    logger = logging.getLogger(name)

   # Set the overall logging level
    logger.setLevel(logging.INFO)

    if len(logger.handlers) == 0:
        
        # Create handlers
        file_handler = logging.FileHandler(filename)
        stream_handler = logging.StreamHandler()
        
        # Set logging levels for handlers
        file_handler.setLevel(logging.INFO)
        stream_handler.setLevel(logging.INFO)
        
        # Create formatters and add them to handlers
        formatter = logging.Formatter('%(asctime)s | %(filename)s | %(levelname)s | %(message)s')
        file_handler.setFormatter(formatter)
        stream_handler.setFormatter(formatter)
        
        # Add handlers to the logger
        logger.addHandler(file_handler)
        logger.addHandler(stream_handler)

    return logger

def run_analysis(asset, w, logger, temperature=0.7, model_name='gpt-4o'):
    """
    Run LLM-based time series analysis with configurable parameters
    
    Args:
        asset (str): Asset name for the Excel file
        w (int): Window size
        logger (logging.Logger): A logger for file persistence
        temperature (float): Temperature parameter for the model (0.0 to 1.0)
        model_name (str): Model name (e.g., 'gpt-4o', 'gpt-3.5-turbo', etc.)
    """
    # Define constants
    n = 128
    steps = 8
    window = w + steps
    max_retries = 50
    save_interval = 1

    # Define model hyperparameters
    base_hypers = dict(
        temp=temperature,
        alpha=0.95,
        beta=0.3,
        basic=False,
        settings=SerializerSettings(base=10, prec=2, signed=True, time_sep=', ', bit_sep='', minus_sign='-')
    )

    # Define available models and their configurations
    model_hypers = {
        f'LLMTime {model_name}': {'model': model_name, **base_hypers}
    }

    model_predict_fns = {
        f'LLMTime {model_name}': get_llmtime_predictions_data
    }

    model_names = list(model_predict_fns.keys())
    current_model_key = f'LLMTime {model_name}'

    # Create unique filenames based on parameters
    results_filename = f"rolling_results_{asset}_w{w}_temp{temperature}_{model_name.replace('-', '_')}.pkl"
    time_log_filename = f"time_log_{asset}_w{w}_temp{temperature}_{model_name.replace('-', '_')}.pkl"
    
    # Delete existing pkl files at the beginning
    if os.path.exists(results_filename):
        os.remove(results_filename)
    if os.path.exists(time_log_filename):
        os.remove(time_log_filename)

    # Load the DataFrame
    df = pd.read_excel(f"datasets/{asset}.xlsx")
    df['Date'] = pd.to_datetime(df['Date'])

    df = df[df['Date'] >= "2021-10-01"]
    df = df.set_index('Date')
    df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
    df = df[['Log_Return']]
    df = df.dropna()

    def save_results(results, filename=results_filename):
        with open(filename, "wb") as f:
            pickle.dump(results, f)

    def load_results(filename=results_filename):
        try:
            with open(filename, "rb") as f:
                return pickle.load(f)
        except FileNotFoundError:
            return []

    def save_time_log(start_time, filename=time_log_filename):
        with open(filename, "wb") as f:
            pickle.dump(start_time, f)

    def load_time_log(filename=time_log_filename):
        try:
            with open(filename, "rb") as f:
                return pickle.load(f)
        except FileNotFoundError:
            return None

    def get_dataset_DP(df):
        series = df.iloc[:, 0]
        return series

    def get_datasets_DP(ds, ds_name, testfrac=0.2, predict_steps=steps):
        series = get_dataset_DP(ds)
        splitpoint = len(series) - predict_steps if predict_steps is not None else int(len(series) * (1 - testfrac))
        train = series.iloc[:splitpoint]
        test = series.iloc[splitpoint:]
        return {ds_name: (train, test)}

    # Main processing function
    def rolling_window_datasets(df, window_length=window, predict_steps=steps):
        rolling_results = load_results()
        start_index = len(rolling_results)
        total_iterations = len(df) - window_length

        for start in range(start_index, total_iterations):
            try:
                logger.info(f"Processing window starting at index {start} of {total_iterations} (Asset: {asset}, Window: {w}, Temp: {temperature}, Model: {model_name})")
                end = start + window_length
                ds = df.iloc[start:end]
                datasets = get_datasets_DP(ds, 'ds', predict_steps=steps)
                ds_name = 'ds'
                train, test = datasets[ds_name]

                out = {}
                dates = ds.index[-1]
                last_value = ds.iloc[-1]

                for model in model_names:
                    random_seed = np.random.randint(0, 100000)
                    model_hypers[model].update({'dataset_name': ds_name, 'random_seed': random_seed})
                    hypers = list(grid_iter(model_hypers[model]))

                    retries = 0
                    while retries < max_retries:
                        try:
                            pred_dict = get_autotuned_predictions_data(train, test, hypers, n, model_predict_fns[model], verbose=False, parallel=True)
                            out[model] = pred_dict['samples']
                            break
                        except OpenAIError as e:
                            retries += 1
                            logger.error(f"API error (attempt {retries}/{max_retries}): {e}")
                            sleep(1)
                            if retries == max_retries:
                                logger.error(f"Failed to get predictions for {model} after {max_retries} retries.")
                                out[model] = np.nan

                rolling_results.append((dates, out, last_value))

                if (start - start_index + 1) % save_interval == 0:
                    logger.info(f"Saving intermediate results at index {start}")
                    save_results(rolling_results)

            except Exception as e:
                logger.error(f"Error at index {start}: {e}")
                traceback.print_exc()
                save_results(rolling_results)
                raise e

        save_results(rolling_results)
        return rolling_results

    # Main loop to handle restarting from the last save
    while True:
        try:
            start_time = time()
            save_time_log(start_time)
            rolling_results = rolling_window_datasets(df)
            end_time = time()
            logger.info(f"Processing time: {end_time - start_time} seconds")

            # Clean up temporary files
            if os.path.exists(results_filename):
                os.remove(results_filename)
            if os.path.exists(time_log_filename):
                os.remove(time_log_filename)
            break
        except Exception as e:
            logger.error("An error occurred. Restarting from last save...")
            sleep(2)

    # Post-processing and output
    prediction_dict = rolling_results
    m = n * steps
    
    # Updated column names to include metadata
    prediction_columns = [f'Log Return {i+1}' for i in range(m)]
    column_names = ['Date', 'Asset', 'Model', 'Temperature', 'Window'] + prediction_columns
    output = pd.DataFrame(columns=column_names)

    # Loop through each item in the predictions dictionary
    for item in prediction_dict:
        date = item[0]  # Extract the date from DatetimeIndex
        data_matrix = item[1].get(current_model_key, None)  # Extract the DataFrame

        if data_matrix is not None and isinstance(data_matrix, pd.DataFrame):
            # Flatten the DataFrame to a single list
            flat_list = data_matrix.values.flatten().tolist()
            
            # Create row with metadata and predictions
            row_data = [date, asset, model_name, temperature, w] + flat_list
            
            # Append this list as a row in the output DataFrame
            output.loc[len(output)] = row_data
        else:
            print(f"Warning: Expected DataFrame at {date} but got {type(data_matrix)}")

    # Write DataFrame to Excel with updated filename
    current_date = datetime.now().strftime("%Y-%m-%d")
    output_filename = f"{current_date}_{asset}_LLMTime_{model_name.replace('-', '_')}_w{w}_temp{temperature}.xlsx"
    output.to_excel(output_filename, index=False)
    
    logger.info(f"Results saved to: {output_filename}")
    return output

def run_temperature_sweep(asset, window_sizes, logger, models=['gpt-4o'], temp_range=(0.0, 1.0, 0.1)):
    """
    Run analysis across multiple temperature values, models, and window sizes
    
    Args:
        asset (str): Asset name
        window_sizes (list): List of window sizes to test
        logger (logging.Logger): Logger for file persistence
        models (list): List of model names to test
        temp_range (tuple): (start, stop, step) for temperature range
    """
    temperatures = np.arange(temp_range[0], temp_range[1] + temp_range[2], temp_range[2])
    temperatures = np.round(temperatures, 1)  # Round to avoid floating point issues
    
    all_results = []
    
    for model_name in models:
        for w in window_sizes:
            for temp in temperatures:
                logger.info(f"Running analysis: Asset={asset}, Model={model_name}, Window={w}, Temperature={temp}")
                
                try:
                    result = run_analysis(asset, w, logger=logger, temperature=temp, model_name=model_name)
                    all_results.append(result)
                    
                    # Optional: save individual results
                    logger.info(f"Completed: {asset}, {model_name}, w={w}, temp={temp}")
                    
                except Exception as e:
                    logger.error(f"Failed for {asset}, {model_name}, w={w}, temp={temp}: {e}")
                    continue
    
    # Combine all results into a single DataFrame
    if all_results:
        combined_results = pd.concat(all_results, ignore_index=True)
        current_date = datetime.now().strftime("%Y-%m-%d")
        combined_filename = f"{current_date}_{asset}_combined_results.xlsx"
        combined_results.to_excel(combined_filename, index=False)
        logger.info(f"All results combined and saved to: {combined_filename}")
        return combined_results
    else:
        print("No results to combine.")
        return None

# Multi-asset, multi-model grid search function
def multi_asset_multi_model_grid_search(assets, models=['gpt-4o'], window_sizes=None, temperatures=None):
    """
    Run complete grid search over multiple assets, models, temperatures and window sizes
    
    Args:
        assets (list): List of asset names (e.g., ['AAPL', 'GOOGL', 'MSFT'])
        models (list): List of model names (e.g., ['gpt-4o', 'gpt-3.5-turbo', 'gpt-4'])
        window_sizes (list): Window sizes (default: [30, 45, 60, 90, 120, 150])
        temperatures (list): Temperature values (default: [0.0, 0.1, ..., 1.0])
    """
    if window_sizes is None:
        window_sizes = [45]
    if temperatures is None:
        temperatures = [round(temp * 0.1, 1) for temp in range(11)]  # [0.0, 0.1, 0.2, ..., 1.0]
    logger = create_logger("gpt4turbo_0.9_1_temp.log", "temperature_sweep")
    
    total_runs = len(assets) * len(models) * len(window_sizes) * len(temperatures)
    current_run = 0
    
    logger.info(f"🚀 Starting Multi-Asset Multi-Model Grid Search")
    logger.info(f"Assets: {assets}")
    logger.info(f"Models: {models}")
    logger.info(f"Window Sizes: {window_sizes}")
    logger.info(f"Temperatures: {temperatures}")
    logger.info(f"Total Runs: {total_runs}")
    
    all_results = []
    failed_runs = []
    
    for asset in assets:
        logger.info(f"💰 Processing Asset: {asset}")
        
        asset_results = []
        
        for model_name in models:
            logger.info(f"🤖 Asset: {asset} | Model: {model_name}")
            
            model_results = []
            
            for window_size in window_sizes:
                logger.info(f"📊 {asset} | {model_name} | Window: {window_size}")
                
                for temp in temperatures:
                    current_run += 1
                    progress = (current_run / total_runs) * 100
                    
                    logger.info(f"[{current_run}/{total_runs}] ({progress:.1f}%) - {asset}|{model_name}|W:{window_size}|T:{temp}")
                    
                    try:
                        result = run_analysis(asset, window_size, logger=logger, temperature=temp, model_name=model_name)
                        all_results.append(result)
                        asset_results.append(result)
                        model_results.append(result)
                        logger.info(f"✅ SUCCESS")
                        
                    except Exception as e:
                        failed_runs.append((asset, model_name, window_size, temp, str(e)))
                        logger.error(f"❌ FAILED - Error: {e}")
                        continue
            
            # Save individual model results for this asset
            if model_results:
                model_combined = pd.concat(model_results, ignore_index=True)
                current_date = datetime.now().strftime("%Y-%m-%d")
                model_filename = f"{current_date}_{asset}_{model_name.replace('-', '_')}_COMPLETE_GRID.xlsx"
                model_combined.to_excel(model_filename, index=False)
                logger.error(f"💾 {asset}-{model_name} results saved to: {model_filename}")
        
        # Save individual asset results (all models)
        if asset_results:
            asset_combined = pd.concat(asset_results, ignore_index=True)
            current_date = datetime.now().strftime("%Y-%m-%d")
            asset_filename = f"{current_date}_{asset}_ALL_MODELS_COMPLETE_GRID.xlsx"
            asset_combined.to_excel(asset_filename, index=False)
            logger.info(f"💾 Asset {asset} (all models) results saved to: {asset_filename}")
    
    # Combine all results across all assets and models
    if all_results:
        combined_results = pd.concat(all_results, ignore_index=True)
        current_date = datetime.now().strftime("%Y-%m-%d")
        
        # Create master file with all assets and models
        master_filename = f"{current_date}_ALL_ASSETS_ALL_MODELS_MASTER_GRID.xlsx"
        combined_results.to_excel(master_filename, index=False)
        
        logger.info(f"🎉 MULTI-ASSET MULTI-MODEL GRID SEARCH COMPLETED!")
        logger.info(f"📁 Master file saved to: {master_filename}")
        
        # Print detailed summary
        logger.info(f"\n📈 FINAL SUMMARY:")
        logger.info(f"- Assets: {len(assets)} ({assets})")
        logger.info(f"- Models: {len(models)} ({models})")
        logger.info(f"- Window Sizes: {len(window_sizes)} ({window_sizes})")
        logger.info(f"- Temperatures: {len(temperatures)} ({min(temperatures)} to {max(temperatures)})")
        logger.info(f"- Total Attempted: {total_runs}")
        logger.info(f"- Successful Runs: {len(all_results)}")
        logger.info(f"- Failed Runs: {len(failed_runs)}")
        logger.info(f"- Success Rate: {(len(all_results)/total_runs)*100:.1f}%")
        logger.info(f"- Total Predictions: {len(combined_results)}")
        
        # Summary by asset and model
        logger.info(f"📊 RESULTS BY ASSET & MODEL:")
        for asset in assets:
            logger.info(f"  {asset}:")
            for model in models:
                asset_model_data = combined_results[
                    (combined_results['Asset'] == asset) & 
                    (combined_results['Model'] == model)
                ]
                expected_runs = len(window_sizes) * len(temperatures)
                actual_runs = len(asset_model_data)
                logger.info(f"    {model}: {actual_runs}/{expected_runs} runs ({(actual_runs/expected_runs)*100:.1f}%)")
        
        if failed_runs:
            logger.error(f"\n❌ Failed Runs Details:")
            for asset, model, window_size, temp, error in failed_runs:
                logger.error(f"   - {asset}|{model}|W:{window_size}|T:{temp} - {error}")
        
        return combined_results
    else:
        logger.error("❌ No successful results to combine.")
        return None

# Multi-asset grid search function (updated to use single model)
def multi_asset_grid_search(assets, model_name='gpt-4o'):
    """
    Run complete grid search over multiple assets, all temperatures and window sizes (single model)
    
    Args:
        assets (list): List of asset names (e.g., ['AAPL', 'GOOGL', 'MSFT'])
        model_name (str): Model name (default: 'gpt-4o')
    """
    return multi_asset_multi_model_grid_search(assets, models=[model_name])

# Example usage:
if __name__ == "__main__":
    # Define your assets and models
    assets = ['SP500']
    models = ['gpt-4-turbo']  # Add your desired models
    
    # Option 1: Multi-asset, multi-model grid search (FULL GRID)
    # For 5 assets × 3 models × 6 windows × 11 temps = 990 total runs
    multi_asset_multi_model_grid_search(assets=assets, models=models, temperatures=[1.])
    
    # Option 2: Multi-asset, single model (original function)
    # For 5 assets × 6 windows × 11 temps = 330 total runs
    # multi_asset_grid_search(assets=assets, model_name='gpt-4o')
    
    # Option 3: Custom parameters
    # multi_asset_multi_model_grid_search(
    #     assets=['AAPL', 'GOOGL'], 
    #     models=['gpt-4o', 'gpt-3.5-turbo'],
    #     window_sizes=[30, 60, 90],
    #     temperatures=[0.0, 0.5, 1.0]
    # )

In [ ]:
import pickle

with open("rolling_results_SP500_w45_temp0.1_gpt_4_turbo.pkl", "rb") as f:
    current_temp0 = pickle.load(f)

In [ ]:
current_temp0[-1]

In [ ]:
import tiktoken

In [ ]:
def read_dataset(asset: str):
    df = pd.read_excel(f"datasets/{asset}.xlsx")
    df['Date'] = pd.to_datetime(df['Date'])

    df = df[df['Date'] >= "2021-10-01"]
    df = df.set_index('Date')
    df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
    df = df[['Log_Return']]
    return df.dropna()

In [ ]:
SP500 = read_dataset("SP500")

In [ ]:
SP500.iloc[:150]

In [ ]:
from data.serialize import serialize_arr

In [ ]:
serialize_arr(SP500["Log_Return"].values * 10_000, SerializerSettings(base=10, prec=2, signed=True, time_sep=', ', bit_sep='', minus_sign='-'))

In [ ]:
import tiktoken

tokenier = tiktoken.get_encoding("cl")

In [ ]:
", ".join(map(str, range(100, 250)))